In [9]:
@file:DependsOn("ro.jf.funds:funds-notebook-client:1.0.0")
%use dataframe
%use kandy

import ro.jf.funds.client.notebook.*
import ro.jf.funds.reporting.api.model.*

// TODO(Johann) this notebook should also contain "Other income".
// This would require to reexport wallet data.
// Could be renamed to income

val username = "Johann-36.0"

val monthlyReportFrom = YearMonthTO(2019, 1)
val monthlyReportTo = YearMonthTO(2021, 12)
val monthlyReportForecastUntil = YearMonthTO(2022, 6)

val yearlyReportFrom = 2019
val yearlyReportTo = 2021
val yearlyReportForecastUntil = 2023

val fundName = "Work Income"
val reportViewName = "Work income report"

val REPORT_DATA_CONFIGURATION_YAML_FILE = "../../data/provision/work-report-data-configuration.yaml"

val client = FundsClient()

In [10]:
val user = client.ensureUserExists(username)
user

UserTO(id=b32eab0a-27f7-4828-aa91-990999a57c37, username=Johann-36.0)

In [11]:
val reportView = client.createReportView(user, reportViewName, fundName, client.fromYaml(File(REPORT_DATA_CONFIGURATION_YAML_FILE), "dataConfiguration"))
reportView

ReportViewTO(id=830dec7d-1507-4fc3-a7cb-5db37bb0e884, name=Work income report, fundId=198f8727-7ab2-404e-9ea8-9a167925e8e5, dataConfiguration=ReportDataConfigurationTO(currency=Currency(value=RON), groups=[ReportGroupTO(name=income, filter=RecordFilterTO(labels=[income])), ReportGroupTO(name=taxes, filter=RecordFilterTO(labels=[work_taxes]))], reports=ReportsConfigurationTO(net=NetReportConfigurationTO(enabled=false, filter=null), valueReport=ValueReportConfigurationTO(enabled=true, filter=null), groupedNet=GenericReportConfigurationTO(enabled=true), groupedBudget=GroupedBudgetReportConfigurationTO(enabled=false, distributions=[])), forecast=ForecastConfigurationTO(inputBuckets=1)))

In [12]:
val monthlyReportData = client.getMonthlyReportViewData(user, reportViewName, monthlyReportFrom, monthlyReportTo, monthlyReportForecastUntil)


In [13]:
val yearlyReportData = client.getYearlyReportViewData(user, reportViewName, yearlyReportFrom, yearlyReportTo, yearlyReportForecastUntil)


In [14]:
import kotlinx.datetime.DateTimeUnit
import kotlinx.datetime.TimeZone
import kotlinx.datetime.atStartOfDayIn
import kotlinx.datetime.minus
import org.jetbrains.kotlinx.kandy.ir.Plot
import java.math.BigDecimal

fun plotGroupedNetData(reportData: ReportDataTO, plotTitle: String, incomeGroupName: String, taxesGroup: String): Plot {
    val dataFrame = dataFrameOf(
        "bucket" to reportData.data.map<ReportDataItemTO, LocalDate> { it.timeBucket.from },
        "income" to reportData.data.map { bucket ->
            bucket.groupedNet!!.firstOrNull { it.group == incomeGroupName }?.net ?: BigDecimal.ZERO
        },
        "taxes" to reportData.data.map { bucket ->
            bucket.groupedNet!!.firstOrNull { it.group == taxesGroup }?.net ?: BigDecimal.ZERO
        },
        "net" to reportData.data.map { bucket ->
            bucket.groupedNet!!.mapNotNull { it.net }.fold(BigDecimal.ZERO) { acc, value -> acc + value }
        },
        "value" to reportData.data.map { bucket ->
            bucket.value!!.end ?: BigDecimal.ZERO
        }
    )

    return dataFrame
        .plot {
            line {
                val values =
                    listOf("net", "taxes", "value").flatMap { dataFrame.get(it).values }.map { it as BigDecimal }
                val forecastBorderMin = minOf(BigDecimal.ZERO, values.minOrNull() ?: BigDecimal.ZERO)
                val forecastBorderMax = values.maxOrNull() ?: BigDecimal.ZERO
                val forecastBorderX = when (reportData.interval.granularity) {
                    TimeGranularityTO.YEARLY -> reportData.interval.toDate.minus(183, DateTimeUnit.DAY)
                    TimeGranularityTO.MONTHLY -> reportData.interval.toDate.minus(15, DateTimeUnit.DAY)
                    TimeGranularityTO.DAILY -> reportData.interval.toDate.minus(1, DateTimeUnit.DAY)
                }

                y(listOf(forecastBorderMin, forecastBorderMax))
                x.constant(forecastBorderX.atStartOfDayIn(TimeZone.UTC).toEpochMilliseconds())
            }
            x("bucket") {
                val format = when (reportData.interval.granularity) {
                    TimeGranularityTO.YEARLY -> "%Y"
                    TimeGranularityTO.MONTHLY -> "%b %Y"
                    TimeGranularityTO.DAILY -> "%d %b %Y"
                }
                axis.breaks(reportData.data.map {
                    it.timeBucket.from.atStartOfDayIn(TimeZone.UTC).toEpochMilliseconds()
                }, format)
            }
            line {
                y.constant(0)
            }
            line {
                y("net")
                color = Color.YELLOW
            }
            line {
                y("taxes")
                color = Color.RED
            }
            line {
                y("income")
                color = Color.GREEN
            }
            area {
                y("value")
                borderLine {
                    color = Color.ORANGE
                }
            }
            layout {
                title = plotTitle
                size = 2400 to 1200
            }
        }

}

In [15]:
plotGroupedNetData(yearlyReportData, "Net income per year", "income", "taxes")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="Q3OLsq"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Net income per year"
},
"mapping":{
},
"data":{
"bucket":[1.5463008E12,1.5778368E12,1.6094592E12,1.6409952E12,1.6725312E12],
"income":[107567.0,138677.75,213978.33,153407.6933333333,153407.6933333333],
"taxes":[0.0,0.0,-10510.52,-3503.506666666667,-3503.506666666667],
"net":[107567.0,138677.75,203467.81,149904.18666666665,149904.18666666665],
"value":[0.0,0.0,46986.81,62649.08,78311.35]
},
"ggsize":{
"width":2400.0,
"height":1200.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5778368E12,1.6094592E12,1.6409952E12,1.6725312E12],
"format":"%Y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5778368E12,1.6094592E12,1.6409952E12,1.6725312E12],
"format":"%Y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5778368E12,1.6094592E12,1.6409952E12,1.6725312E12],
"format":"%Y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5778368E12,1.6094592E12,1.6409952E12,1.6725312E12],
"format":"%Y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5778368E12,1.6094592E12,1.6409952E12,1.6725312E12],
"format":"%Y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"y":"y"
},
"stat":"identity",
"data":{
"y":[-10510.52,203467.81]
},
"sampling":"none",
"x":1.6250976E12,
"inherit_aes":false,
"position":"identity",
"geom":"line"
},{
"mapping":{
"x":"bucket"
},
"stat":"identity",
"sampling":"none",
"y":0.0,
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"bucket",
"y":"net"
},
"stat":"identity",
"color":"#fac858",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"bucket",
"y":"taxes"
},
"stat":"identity",
"color":"#ee6666",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"bucket",
"y":"income"
},
"stat":"identity",
"color":"#3ba272",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"bucket",
"y":"value"
},
"stat":"identity",
"color":"#fc8452",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"area",
"data":{
}
}],
"data_meta":{
"series_annotations":[{
"type":"datetime",
"column":"bucket"
}]
},
"spec_id":"8"
};
 var containerDiv = document.getElementById("Q3OLsq");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 2400.0,
 height: 1200.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 2019 
 
 
 
 
 
 
 
 
 2020 
 
 
 
 
 
 
 
 
 2021 
 
 
 
 
 
 
 
 
 2022 
 
 
 
 
 
 
 
 
 2023 
 
 
 
 
 
 
 
 
 
 
 -20,000 
 
 
 
 
 
 
 -10,000 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 1

In [16]:
plotGroupedNetData(monthlyReportData, "Income per month", "income", "taxes")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="SJR19C"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Income per month"
},
"mapping":{
},
"data":{
"bucket":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12,1.6330464E12,1.6357248E12,1.6383168E12,1.6409952E12,1.6436736E12,1.6460928E12,1.6487712E12,1.6513632E12,1.6540416E12],
"income":[6950.0,7006.0,14784.0,7037.0,8157.0,8227.0,8184.0,8344.0,8007.0,10203.0,10287.0,10381.0,10176.0,10237.0,10229.0,19367.0,11411.0,10040.0,10082.0,10489.0,10110.0,10124.0,11694.0,14718.75,13596.0,13500.0,13595.0,13836.0,14227.0,20886.4,19842.08,18797.76,14620.48,23497.2,24103.77,23476.64,17831.5275,18184.488125,18574.86213541667,18989.85064670139,19419.33820059317,19852.0330506426],
"taxes":[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-500.0,0.0,0.0,-344.81,-1583.32,-2281.51,-345.0,-346.0,-4763.44,-346.44,-875.8766666666667,-948.8663888888889,-1027.938587962963,-1071.93347029321,-1161.261259484311,-1258.033031108003],
"net":[6950.0,7006.0,14784.0,7037.0,8157.0,8227.0,8184.0,8344.0,8007.0,10203.0,10287.0,10381.0,10176.0,10237.0,10229.0,19367.0,11411.0,10040.0,10082.0,10489.0,10110.0,10124.0,11694.0,14718.75,13596.0,13500.0,13095.0,13836.0,14227.0,20541.59,18258.76,16516.25,14275.48,23151.2,19340.33,23130.2,16955.650833333333,17235.621736111112,17546.923547453705,17917.91717640818,18258.07694110886,18594.000019534597],
"value":[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20541.59,2036.35,18552.6,32828.08,4516.28,23856.61,46986.81,50902.3775,55144.24229166667,59739.59581597222,64717.89546730324,70111.05342291185,74241.84204148783]
},
"ggsize":{
"width":2400.0,
"height":1200.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12,1.6330464E12,1.6357248E12,1.6383168E12,1.6409952E12,1.6436736E12,1.6460928E12,1.6487712E12,1.6513632E12,1.6540416E12],
"format":"%b %Y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12,1.6330464E12,1.6357248E12,1.6383168E12,1.6409952E12,1.6436736E12,1.6460928E12,1.6487712E12,1.6513632E12,1.6540416E12],
"format":"%b %Y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.551